In [1]:
#basic package
import json
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import shutil
import time
import datetime as dt
import itertools
import time
import re
from collections import Counter
from collections import defaultdict
import operator
import math

#plot
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from UTILS import time_series_henColumn_tsRow, ZoneVariable, HenVariable, DistributionEntropy, \
heatmap_duration_perzone_perhen, TimeSeriesPlot, entropy_compare_session
import config_mobility as config

we restrict the time series to one value per 60 seconds 
we compute the complexity variables each 30 minutes 
each variables includes the values of at least the last 120.00 minutes (i.e. are using 120.00 values)


In [3]:
path_extracted_data = config.path_extracted_data
id_run = config.id_run
nbr_sec = config.nbr_sec
#day level
ts_name = 'time_serie_day'
print(id_run)

VF


# Download Data

In [4]:
df_event = pd.read_csv(os.path.join(path_extracted_data,'df_event.csv'), sep=';')
print(df_event.shape)
df_event.head(3)

(989, 12)


,HenID,sessionNumber,severity,group,HenID_sessionNumber,previous_sessionNumber,previous_HenID_sessionNumber,previous_severity,sum_previous_severity,severity-previousSeverity,is_interesting_event,is_not_interesting_event
0,hen_1,2,0.0,A,hen_1_2,1.0,hen_1_1,0.0,0.0,0.0,False,True
1,hen_1,3,0.8,A,hen_1_3,2.0,hen_1_2,0.0,0.0,0.8,False,False
2,hen_1,4,0.3,A,hen_1_4,3.0,hen_1_3,0.8,0.8,-0.5,False,False


In [5]:
df_event[df_event['is_interesting_event']].head(3)

,HenID,sessionNumber,severity,group,HenID_sessionNumber,previous_sessionNumber,previous_HenID_sessionNumber,previous_severity,sum_previous_severity,severity-previousSeverity,is_interesting_event,is_not_interesting_event
10,hen_2,4,2.1,A,hen_2_4,3.0,hen_2_3,0.0,0.0,2.1,True,False
25,hen_4,4,1.5,A,hen_4_4,3.0,hen_4_3,0.0,0.0,1.5,True,False
41,hen_6,2,3.0,A,hen_6_2,1.0,hen_6_1,0.0,0.0,3.0,True,False


# Select sub event table based on flag

In [43]:
#option 1: keep all events
#df_sub_event = df_event.copy()

#option 2: keep only interesting event
df_sub_event = df_event[df_event['is_interesting_event']].copy()

#option 2: keep only not interesting event
#df_sub_event = df_event[df_event['is_not_interesting_event']].copy()

#show info
print(df_sub_event.shape)
df_sub_event.head(3)

(78, 12)


,HenID,sessionNumber,severity,group,HenID_sessionNumber,previous_sessionNumber,previous_HenID_sessionNumber,previous_severity,sum_previous_severity,severity-previousSeverity,is_interesting_event,is_not_interesting_event
10,hen_2,4,2.1,A,hen_2_4,3.0,hen_2_3,0.0,0.0,2.1,True,False
25,hen_4,4,1.5,A,hen_4_4,3.0,hen_4_3,0.0,0.0,1.5,True,False
41,hen_6,2,3.0,A,hen_6_2,1.0,hen_6_1,0.0,0.0,3.0,True,False


In [44]:
#decide for a name under which the generated event with variable file will be saved
saving_name_sub = 'interesting_EVENT_VAR' #should not end with _variables

# Keep all variable before and after with no aggregation

li_path = glob.glob(os.path.join(path_extracted_data, id_run+'_'+ts_name+'_*_variables.csv'))
li_parse_var = ['level','FirstTimestamp_zone_1','FirstTimestamp_zone_2', 'FirstTimestamp_zone_3','FirstTimestamp_zone_4', 
                'FirstTimestamp_zone_5']
li_df = []
for path_sess in tqdm.tqdm(li_path):
    df_sess = pd.read_csv(path_sess, sep=';', parse_dates=li_parse_var)
    sessID = path_sess.split('_')[-2]
    #df_sess['SessionID'] = sessID
    df_sess.drop(['list_of_durations','zone_list','dur_values','dur_values_normalized'],inplace=True,axis=1) #SampEnt_perZone
    print(df_sess.shape)
    print(sessID)
    df_sess.to_csv(os.path.join(path_extracted_data, id_run+'_'+ts_name+'_'+sessID+'_variables_SIMPLER.csv'), sep=';')
    #li_df.append(df_sess)
    
#df = pd.concat(li_df)
#print(df.shape)
#df.head(3)

# Aggregate variable from two sessions - event level variables

In [37]:
def Sum1MinusSum2_4dico(d, config):
    #two list of dictionaries and take list of same size by removing the last level
    x0 = [eval(i) for i in d[0]] ; x1 = [eval(i) for i in d[1]]
    m = min(len(x0),len(x1))
    x0 = x0[0:m] ; x1 = x1[0:m]
    return(str({k: sum(v.get(k,np.nan) for v in x0)-
               sum(v.get(k,np.nan) for v in x1) for k in config.dico_zone_order.keys()}))
#small exemple
#x1 = ["{'zone_1': 0, 'zone_2': 0, 'zone_3': 3, 'zone_4': 1, 'zone_5': 7}", 
#      "{'zone_1': 0, 'zone_2': 3, 'zone_3': 1, 'zone_4': 2, 'zone_5': 1}", 
#      "{'zone_1': 0, 'zone_2': 0, 'zone_3': 3, 'zone_4': 2, 'zone_5': 0}"] 
#x2 = ["{'zone_1': 0, 'zone_2': 0, 'zone_3': 7, 'zone_4': 1, 'zone_5': 0}", 
#      "{'zone_1': 0, 'zone_2': 0, 'zone_3': 3, 'zone_4': 2, 'zone_5': 1}", 
#      "{'zone_1': 0, 'zone_2': 0, 'zone_3': 0, 'zone_4': 3, 'zone_5': 1}"]
#Sum1MinusSum2_4dico((x1,x2), config) #{'zone_1': 0, 'zone_2': 3, 'zone_3': -3, 'zone_4': -1, 'zone_5': 6}

def Sum1MinusSum2(x):
    l0 = len(x[0]); l1 = len(x[1])
    return(sum(x[0][0:min(l0,l1)])-sum(x[1][0:min(l0,l1)]))

def MostFrequent(x):
    return(max(Counter(x).items(), key=operator.itemgetter(1))[0])
#small exemple
#MostFrequent(['1','1','4','5','4','5','5','5']) #5

#LEVEL IS DAY NOT ZONE
#x is of the following form (if in this case the hen has 6 days in each of the two compared session):
#x = ([var_sess1_level1, var_sess1_level2, var_sess1_level3, var_sess1_level4, var_sess1_level5, var_sess1_level6],
#     [var_sess2_level1, var_sess2_level2, var_sess2_level3, var_sess2_level4, var_sess2_level5, var_sess2_level6])
#knowing this, one can define the computation that we want
#Note: when we want to compare entry per entry or use an aggregate-fct that depends highly on the number of entries (e.g. sum), 
#then we must pay attention in case some session will have less level/day than other, we will take the same nbr of level into 
#account for both sessions (e.g. Total_number_zone_Sum1MinusSum2, Total_number_transition_SumOfEntry1MinusEntr2)
#Note that SumOfEntry1MinusEntry2 = Sum1MinusSum, but in case of date its better to use the first one
dico_var_agg = {('Variance_duration', 'Variance_duration_before'): lambda x: ' - '.join([str(i) for i in x[0]]),
                ('Variance_duration', 'Variance_duration_after'): lambda x: ' - '.join([str(i) for i in x[1]]),
                ('Variance_duration', 'Variance_duration_Avg1MinusAvg2'): lambda x: np.mean(x[0])-np.mean(x[1]),
                ('Variance_duration', 'Variance_duration_Max1MinusMax2'): lambda x: max(x[0])-max(x[1]),
                ('duration_zone_4', 'duration_zone_4_Sum1MinusSum2'): lambda x: Sum1MinusSum2(x),
                ('Max_duration', 'Max_duration_Max1MinusMax2'): lambda x: max(x[0])-max(x[1]),
                ('Average_duration', 'Average_duration_Max1MinusMax2'): lambda x: max(x[0])-max(x[1]),
                ('Median_duration', 'Median_duration_Max1MinusMax2'): lambda x: max(x[0])-max(x[1]),
                ('Total_number_zone', 'Total_number_zone_Sum1MinusSum2'): lambda x: Sum1MinusSum2(x),
                ('Total_number_transition', 'Total_number_transition_Sum1MinusSum2'): lambda x: Sum1MinusSum2(x),
                ('Total_number_transition', 'Total_number_transition_MaxOfEntry1MinusEntry2'): lambda x: max([x[0][i]-x[1][i] \
                                                                    for i in range(min(len(x[0]),len(x[1])))]),
                ('SampEnt_perZone', 'SampEnt_perZone_Sum1MinusSum2'): lambda x : Sum1MinusSum2_4dico(x, config),
                ('SampEnt_order2', 'SampEnt_order2_LastEntry1MinusLastEntry2'): lambda x: x[0][-1]-x[1][-1],
                ('distribution_entropy', 'distribution_entropy_LastEntry1MinusLastEntry2'): lambda x: x[0][-1]-x[1][-1],
                ('nbr_bouts_zone_1', 'nbr_bouts_zone_1_Sum1MinusSum2'): lambda x: Sum1MinusSum2(x),
                ('FirstTimestamp_zone_3', 'FirstTimestamp_zone_3_SumOfEntry1MinusEntry2'): lambda x: sum([(x[0][i]-x[1][i]).total_seconds() \
                                                                    for i in range(min(len(x[0]),len(x[1])))]),
                ('Max_duration_zones', 'Max_duration_zones_MostFrequent1'): lambda x: MostFrequent(x[0]),
                ('Max_duration_zones', 'Max_duration_zones_MostFrequent2'): lambda x: MostFrequent(x[1]),
                ('RunSampEnt_onLastTsOfEachLevel_5', 'RunSampEnt_onLastTsOfEachLevel_5_Max1MinusMax2'): lambda x: max(x[0])-max(x[1]),
                ('RunSampEnt_onLastTsOfEachLevel_2', 'RunSampEnt_onLastTsOfEachLevel_2_Min1MinusMin2'): lambda x: min(x[0])-min(x[1]),
                ('RunSampEnt_onLastTsOfEachLevel', 'RunSampEnt_onLastTsOfEachLevel_Max1MinusMax2'): lambda x: max(x[0])-max(x[1]),
                ('RunDistEnt_onLastTsOfEachLevel', 'RunDistEnt_onLastTsOfEachLevel_Max1MinusMax2'): lambda x: max(x[0])-max(x[1]),
               }
#small exemple
#dico_var_aggregationType[('Variance_duration', 'Variance_duration_min_sub')](([2,2,3,1,1,2],[3,4,4,4,4,3])) #-2

In [ ]:
df_sub_event = df_sub_event.reset_index(drop=True)
#loop over lal the slected events
for i in tqdm.tqdm(range(df_sub_event.shape[0])):
    

    #initialise info of the event
    x = df_sub_event.iloc[i]
    HenID = x['HenID']
    session1 = str(int(x['previous_sessionNumber'])) + x['group']
    session2 = str(int(x['sessionNumber'])) + x['group']

    #open the sessions-variable csv and select correct hen
    df_sess1 = pd.read_csv(os.path.join(path_extracted_data, id_run+'_'+ts_name+'_'+session1+'_variables.csv'), sep=';', 
                          parse_dates=['level','FirstTimestamp_zone_1','FirstTimestamp_zone_2', 'FirstTimestamp_zone_3', 
                                       'FirstTimestamp_zone_4', 'FirstTimestamp_zone_5'])
    df_sess2 = pd.read_csv(os.path.join(path_extracted_data, id_run+'_'+ts_name+'_'+session2+'_variables.csv'), sep=';', 
                          parse_dates=['level','FirstTimestamp_zone_1','FirstTimestamp_zone_2', 'FirstTimestamp_zone_3', 
                                       'FirstTimestamp_zone_4', 'FirstTimestamp_zone_5'])
    df_sess1 = df_sess1[df_sess1['HenID']==HenID].sort_values(['level'], ascending=True)
    df_sess2 = df_sess2[df_sess2['HenID']==HenID].sort_values(['level'], ascending=True)

    #verify if 6 days per session. Note that due to both combination: 8A 9A ; 9A 10A we might have the warning twice depending 
    #on the selected sub_event
    if df_sess1.shape[0]!=6:
        print('WARNING: hen %s has %d days in session %s'%(HenID, df_sess1.shape[0], session1))
    if df_sess2.shape[0]!=6:
        print('WARNING: hen %s has %d days in session %s'%(HenID, df_sess2.shape[0], session2))

    #create variables
    for v,fct in dico_var_agg.items():
        old_var = v[0] ; new_var = v[1]
        x1 = df_sess1[old_var].tolist() ; x2 = df_sess2[old_var].tolist()
        df_sub_event.loc[(df_sub_event['HenID']==HenID)&(df_sub_event['sessionNumber']==x['previous_sessionNumber']),
                         new_var] = fct((x1,x2))

#save the sub_event table with variables
df_sub_event.to_csv(os.path.join(path_extracted_data, id_run+'_'+ts_name+'_'+saving_name_sub+'.csv'), sep=';', 
                    index=False)
print(df_sub_event.shape)
df_sub_event

 42%|██████████████████████████████████▋                                               | 33/78 [02:17<03:27,  4.60s/it]

In [14]:
df_sub_event[['Variance_duration_before', 'Variance_duration_after']]

,Variance_duration_before,Variance_duration_after
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [70]:
df_sub_event = df_sub_event.reset_index(drop=True)
#loop over lal the slected events
for i in tqdm.tqdm(range(df_sub_event.shape[0])):
    

    #initialise info of the event
    x = df_sub_event.iloc[i]
    HenID = x['HenID']
    session1 = str(int(x['previous_sessionNumber'])) + x['group']
    session2 = str(int(x['sessionNumber'])) + x['group']

    #open the sessions-variable csv and select correct hen
    df_sess1 = pd.read_csv(os.path.join(path_extracted_data, id_run+'_'+ts_name+'_'+session1+'_variables.csv'), sep=';', 
                          parse_dates=['level','FirstTimestamp_zone_1','FirstTimestamp_zone_2', 'FirstTimestamp_zone_3', 
                                       'FirstTimestamp_zone_4', 'FirstTimestamp_zone_5'])
    df_sess2 = pd.read_csv(os.path.join(path_extracted_data, id_run+'_'+ts_name+'_'+session2+'_variables.csv'), sep=';', 
                          parse_dates=['level','FirstTimestamp_zone_1','FirstTimestamp_zone_2', 'FirstTimestamp_zone_3', 
                                       'FirstTimestamp_zone_4', 'FirstTimestamp_zone_5'])
    df_sess1 = df_sess1[df_sess1['HenID']==HenID].sort_values(['level'], ascending=True)
    df_sess2 = df_sess2[df_sess2['HenID']==HenID].sort_values(['level'], ascending=True)

    #verify if 6 days per session. Note that due to both combination: 8A 9A ; 9A 10A we might have the warning twice depending 
    #on the selected sub_event
    if df_sess1.shape[0]!=6:
        print('WARNING: hen %s has %d days in session %s'%(HenID, df_sess1.shape[0], session1))
    if df_sess2.shape[0]!=6:
        print('WARNING: hen %s has %d days in session %s'%(HenID, df_sess2.shape[0], session2))

    #create variables
    for v,fct in dico_var_agg.items():
        old_var = v[0] ; new_var = v[1]
        x1 = df_sess1[old_var].tolist() ; x2 = df_sess2[old_var].tolist()
        df_sub_event.loc[(df_sub_event['HenID']==HenID)&(df_sub_event['sessionNumber']==x['previous_sessionNumber']),
                         new_var] = fct((x1,x2))

#save the sub_event table with variables
df_sub_event.to_csv(os.path.join(path_extracted_data, id_run+'_'+ts_name+'_'+saving_name_sub+'_variables.csv'), sep=';', 
                    index=False)
print(df_sub_event.shape)
df_sub_event





  0%|                                                                                          | 0/989 [00:00<?, ?it/s]



  0%|                                                                                | 1/989 [00:03<1:05:00,  3.95s/it]



  0%|▏                                                                               | 2/989 [00:07<1:04:40,  3.93s/it]



  0%|▏                                                                               | 3/989 [00:11<1:04:13,  3.91s/it]



  0%|▎                                                                               | 4/989 [00:15<1:03:35,  3.87s/it]



  1%|▍                                                                               | 5/989 [00:19<1:03:03,  3.85s/it]



  1%|▍                                                                               | 6/989 [00:23<1:02:40,  3.83s/it]



  1%|▌                                                                               | 7/989 [00:26<1:01:07,  3.73s/it]



  1%|▋      

 13%|██████████▋                                                                     | 132/989 [07:31<51:59,  3.64s/it]



 13%|██████████▊                                                                     | 133/989 [07:34<52:26,  3.68s/it]



 14%|██████████▊                                                                     | 134/989 [07:38<51:06,  3.59s/it]



 14%|██████████▉                                                                     | 135/989 [07:41<49:16,  3.46s/it]



 14%|███████████                                                                     | 136/989 [07:45<50:27,  3.55s/it]



 14%|███████████                                                                     | 137/989 [07:48<51:51,  3.65s/it]



 14%|███████████▏                                                                    | 138/989 [07:52<52:33,  3.71s/it]



 14%|███████████▏                                                                    | 139/989 [07:56<52:27,  3.70s/it]



 14%|███████████

 27%|█████████████████████▎                                                          | 264/989 [14:58<41:12,  3.41s/it]



 27%|█████████████████████▍                                                          | 265/989 [15:01<40:50,  3.38s/it]



 27%|█████████████████████▌                                                          | 266/989 [15:04<40:25,  3.35s/it]



 27%|█████████████████████▌                                                          | 267/989 [15:08<40:14,  3.34s/it]



 27%|█████████████████████▋                                                          | 268/989 [15:10<38:45,  3.23s/it]



 27%|█████████████████████▊                                                          | 269/989 [15:14<39:39,  3.30s/it]



 27%|█████████████████████▊                                                          | 270/989 [15:17<39:57,  3.33s/it]



 27%|█████████████████████▉                                                          | 271/989 [15:21<40:14,  3.36s/it]



 28%|███████████





 28%|██████████████████████▏                                                         | 274/989 [15:30<38:41,  3.25s/it]





 28%|██████████████████████▏                                                         | 275/989 [15:34<38:25,  3.23s/it]



 28%|██████████████████████▎                                                         | 276/989 [15:37<39:47,  3.35s/it]



 28%|██████████████████████▍                                                         | 277/989 [15:41<40:22,  3.40s/it]



 28%|██████████████████████▍                                                         | 278/989 [15:44<40:23,  3.41s/it]



 28%|██████████████████████▌                                                         | 279/989 [15:48<40:02,  3.38s/it]



 28%|██████████████████████▋                                                         | 280/989 [15:51<40:20,  3.41s/it]



 28%|██████████████████████▋                                                         | 281/989 [15:54<40:03,  3.39s/it]



 29%|██████████████████████▊                                                         | 282/989 [15:58<39:18,  3.34s/it]



 29%|███████

...





 99%|███████████████████████████████████████████████████████████████████████████████▏| 979/989 [58:02<00:34,  3.47s/it]



 99%|███████████████████████████████████████████████████████████████████████████████▎| 980/989 [58:06<00:32,  3.60s/it]



 99%|███████████████████████████████████████████████████████████████████████████████▎| 981/989 [58:10<00:29,  3.74s/it]



 99%|███████████████████████████████████████████████████████████████████████████████▍| 982/989 [58:14<00:26,  3.79s/it]



 99%|███████████████████████████████████████████████████████████████████████████████▌| 983/989 [58:18<00:22,  3.82s/it]



 99%|███████████████████████████████████████████████████████████████████████████████▌| 984/989 [58:22<00:19,  3.83s/it]



100%|███████████████████████████████████████████████████████████████████████████████▋| 985/989 [58:26<00:15,  3.82s/it]



100%|███████████████████████████████████████████████████████████████████████████████▊| 986/989 [58:29<00:11,  3.83s/it]



100%|███████





100%|███████████████████████████████████████████████████████████████████████████████▉| 988/989 [58:36<00:03,  3.56s/it]





100%|████████████████████████████████████████████████████████████████████████████████| 989/989 [58:39<00:00,  3.34s/it]

(989, 32)


,HenID,sessionNumber,severity,group,HenID_sessionNumber,previous_sessionNumber,previous_HenID_sessionNumber,previous_severity,sum_previous_severity,severity-previousSeverity,...,SampEnt_order2_LastEntry1MinusLastEntry2,distribution_entropy_LastEntry1MinusLastEntry2,nbr_bouts_zone_1_Sum1MinusSum2,Max_duration_zones_MostFrequent1,Max_duration_zones_MostFrequent2,RunSampEnt_onLastTsOfEachLevel_5_Max1MinusMax2,RunSampEnt_onLastTsOfEachLevel_2_Min1MinusMin2,RunSampEnt_onLastTsOfEachLevel_Max1MinusMax2,RunDistEnt_onLastTsOfEachLevel_Max1MinusMax2,FirstTimestamp_zone_3_SumOfEntry1MinusEntry2
0,hen_1,2,0.0,A,hen_1_2,1.0,hen_1_1,0.0,0.0,0.0,...,0.000233,0.154802,41.0,zone_2,zone_2,0.005822,-0.010833,-0.013691,0.173485,-1.08804e+07
1,hen_1,3,0.8,A,hen_1_3,2.0,hen_1_2,0.0,0.0,0.8,...,0.001391,0.215233,7.0,zone_2,zone_5,-0.055727,0.170565,0.102237,-0.034236,-1.45861e+07
2,hen_1,4,0.3,A,hen_1_4,3.0,hen_1_3,0.8,0.8,-0.5,...,-0.001577,0.136367,10.0,zone_5,zone_5,-0.027490,0.020912,0.003255,0.470362,NaN
3,hen_1,5,0.2,A,hen_1_5,4.0,hen_1_4,0.3,1.1,-0.1,...,0.001512,0.040120,0.0,zone_5,zone_5,0.039015,0.000000,0.039929,0.081746,NaN
4,hen_1,6,1.5,A,hen_1_6,5.0,hen_1_5,0.2,1.3,1.3,...,0.001645,0.677761,0.0,zone_5,zone_5,-0.035711,0.000000,-0.037691,-0.041703,NaN
5,hen_1,7,1.6,A,hen_1_7,6.0,hen_1_6,1.5,2.8,0.1,...,-0.004498,-0.656555,0.0,zone_5,zone_5,0.017596,0.000000,0.019380,-0.018453,NaN
6,hen_1,8,1.5,A,hen_1_8,7.0,hen_1_7,1.6,4.4,-0.1,...,0.003734,0.154004,0.0,zone_5,zone_5,0.042567,0.000000,0.042762,0.196984,NaN
7,hen_1,9,1.5,A,hen_1_9,8.0,hen_1_8,1.5,5.9,0.0,...,0.000009,-0.088225,0.0,zone_5,zone_5,0.005966,0.000000,0.005741,-0.063519,NaN
8,hen_1,10,1.8,A,hen_1_10,9.0,hen_1_9,1.5,7.4,0.3,...,-0.000138,0.009582,0.0,zone_5,zone_5,0.008698,0.000000,0.008900,0.045539,NaN
9,hen_1,11,1.6,A,hen_1_11,10.0,hen_1_10,1.8,9.2,-0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


#in case to try things
session1 = '10B'
df_sess1 = pd.read_csv(os.path.join(path_extracted_data, id_run+'_'+ts_name+'_'+session1+'_variables.csv'), sep=';', 
                      parse_dates=['level','FirstTimestamp_zone_1','FirstTimestamp_zone_2', 'FirstTimestamp_zone_3', 
                                   'FirstTimestamp_zone_4', 'FirstTimestamp_zone_5'])

df_sess1[df_sess1['HenID']=='hen_85'][['FirstTimestamp_zone_1','FirstTimestamp_zone_2', 'FirstTimestamp_zone_3', 
                                   'FirstTimestamp_zone_4', 'FirstTimestamp_zone_5']]